In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from astropy import wcs

import dask, xarray
#print('dask',dask.__version__, 'xarray',xarray.__version__)

import xarray as xr

def get_image_coords(fits_header):
    w = wcs.WCS(fits_header)
    NAXIS1 = fits_header["NAXIS1"]
    NAXIS2 = fits_header["NAXIS2"]
    x = np.arange(NAXIS1)
    y = np.arange(NAXIS2)
    X, Y = np.meshgrid(x, y)
    return w.wcs_pix2world(X, Y, 0)  #returns ra, dec arrays.



DEFAULT_RES = 0
IMAGE_SPECTRAL_RES = 6  #number of image bands
xr.set_options(display_style="html")
img, img_err
x_res = img.shape[1]
y_res = img.shape[0]

ra_coords, dec_coords = get_image_coords(fits_header)
print (ra_coords.shape)
date_time_str = ("%s %s" %(fits_header["DATE-OBS"], fits_header["TAIHMS"]))
image_date_time_obs = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
band = fits_header["FILTER"]
wavelength = filter_midpoints[band]
wl_res = 6 

img_6d = np.array (img) [:, :,  np.newaxis, np.newaxis, np.newaxis, np.newaxis, np.newaxis]
img_err_6d = np.array (img_err) [:, :,  np.newaxis, np.newaxis, np.newaxis, np.newaxis, np.newaxis]
print(img_6d.shape)

img_ds = xr.Dataset({'flux_mean': (['x', 'y', 'date_time', 'wavelength', 'x_res', 'y_res', 'wl_res'], img_6d),
                 'flux_variance': (['x', 'y', 'date_time', 'wavelength', 'x_res', 'y_res', 'wl_res'], img_err_6d)},
                coords={'ra': (['x', 'y'], ra_coords),
                        'dec': (['x', 'y'], dec_coords),
                        'date_time': [image_date_time_obs],
                        'wavelength': [wavelength],
                        'x_res': [x_res], 
                        'y_res': [y_res], 
                        'wl_res': [IMAGE_SPECTRAL_RES]})



img_ds.to_netcdf(path="test_image_cdf.nc")
#from matplotlib.colors import LogNorm



#plt.imshow(fits_data, cmap='gray', norm=LogNorm())
#plt.colorbar()

img_ds




In [ ]:
flux_var_nmfrom astropy import time

wl, flux, flux_err

plate_obs_date_time = time.Time(spectrum_header["MJD"], format='mjd', out_subfmt='date_hms').datetime
print(plate_obs_date_time)
ra = [[spectrum_header['PLUG_RA']]]     # to match the image-like x,y coordinates
dec = [[spectrum_header['PLUG_DEC']]]   # to match the image-like x,y coordinates
wavelength = wl
wl_res = len(wl)
x_res = 1    #only one object observed
y_res = 1    #only one object observed

spec_6d = np.array (flux) [np.newaxis, np.newaxis, np.newaxis, :, np.newaxis, np.newaxis, np.newaxis]
spec_err_6d = np.array (flux_err) [np.newaxis, np.newaxis, np.newaxis, :, np.newaxis, np.newaxis, np.newaxis]
print(spec_6d.shape)

spec_ds = xr.Dataset({'flux_mean': (['x', 'y', 'date_time', 'wavelength', 'x_res', 'y_res', 'wl_res'], spec_6d),
                 'flux_variance': (['x', 'y', 'date_time', 'wavelength', 'x_res', 'y_res', 'wl_res'], spec_err_6d)},
                coords={'ra': (['x', 'y'], ra),
                        'dec': (['x', 'y'], dec),
                        'date_time': [plate_obs_date_time],
                        'wavelength': wavelength,
                        'x_res': [x_res], 
                        'y_res': [y_res], 
                        'wl_res': [wl_res]})



spec_ds.to_netcdf(path="test_spectrum_cdf.nc")
#from matplotlib.colors import LogNorm



#plt.imshow(fits_data, cmap='gray', norm=LogNorm())
#plt.colorbar()

spec_ds

In [ ]:
xarray.concat(spec_ds.sizes['wavelength'] * [img_ds], dim='wavelength')


In [ ]:
db_setup_sql = "setupDB.sql"
db_connector = DBConnector("localhost", 7001, "rasadmin", "rasadmin")
query_executor = QueryExecutor(db_connector)
db_connector.open()

query_drop1 = r"""drop collection cube""";
query_drop2 = r"""drop type GaussianMeasurementflux_var_nm""";
query_drop3 = r"""drop type GaussianCube""";
query_drop4 = r"""drop type GaussianMeasurement""";


query1 = r"""create type GaussianMeasurement
	as ( mean float, 
		variance float)""";

query2 = r"""create type GaussianCube 
	as GaussianMeasurement mdarray [ra, dec, time, spectral, res]""";

query3 = r"""create type GaussianCubeSet 
	as set (GaussianCube)""";

query4 = r"""create collection cube GaussianCubeSet""";

query5 = r"""select r from RAS_COLLECTIONNAMES as r """;


with open(db_setup_sql, 'r') as file:
    query = file.read()
try:
    
    #query_executor.execute_write(query_drop1)
    #query_executor.execute_write(query_drop2)
    #query_executor.execute_write(query_drop3)
    #query_executor.execute_write(query_drop4)
    #query_executor.execute_write(query1)
    query_executor.execute_write(query2)
    query_executor.execute_write(query3)
    query_executor.execute_write(query4)
    result = query_executor.execute_read(query5)
    
    
    
    #numpy_array = result.to_array()
    print(result)
finally:
    db_connector.close()



In [ ]:
import numpy as np
from astropy import wcs
from astropy.io import fits
import sys
from astropy.coordinates import SkyCoord


w = wcs.WCS(fits_header)
print (w)
ra_left, dec_top = w.wcs_pix2world(0, 0, 1)

ra_right, dec_bot = w.all_pix2world(fits_header["NAXIS1"], fits_header["NAXIS2"], 1)

print ("px: %f py: %f" %(ra_left, dec_top ))
print ("p2x: %f p2y: %f" %(ra_right, dec_bot))


coord_top_left = SkyCoord.from_pixel(0,0,w)
#coord_top_left.representation_type = 'cartesian'
print(coord_top_left)

coord_bot_left = SkyCoord.from_pixel(0,fits_header["NAXIS2"],w)
coord_bot_left.representation_type = 'cartesian'
print(coord_bot_left)

coord_top_right = SkyCoord.from_pixel(fits_header["NAXIS1"],0,w)
coord_top_right.representation_type = 'cartesian'
print(coord_top_right)

coord_bot_right = SkyCoord.from_pixel(fits_header["NAXIS1"],fits_header["NAXIS2"],w)
coord_bot_right.representation_type = 'cartesian'
print(coord_bot_right)


ra_left, dec_top = w.wcs_pix2world(0, 0, 1)
ra_right, dec_bot = w.all_pix2world(fits_header["NAXIS1"], fits_header["NAXIS2"], 1)

print ("px: %f py: %f" %(ra_left, dec_top ))
print ("p2x: %f p2y: %f" %(ra_right, dec_bot))

In [ ]:
import healpy as hp
import datetime
import numpy as np

RES_ORDER = 19
NSIDE = 2**RES_ORDER
NPIX = hp.nside2npix(NSIDE)



time_epoch = fits_header["TAI"]
band = fits_header["FILTER"]
spec = filter_midpoints[band]
res = RES_ORDER

date_time = ("%s %s" %(fits_header["DATE-OBS"], fits_header["TAIHMS"]))
datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S.%f')



NAXIS1 = fits_header["NAXIS1"]
NAXIS2 = fits_header["NAXIS2"]
x = np.arange(NAXIS1)
y = np.arange(NAXIS2)
X, Y = np.meshgrid(x, y)
ra, dec = w.wcs_pix2world(X, Y, 0)




fig = plt.figure(num=None, figsize=(20, 14), dpi=80, facecolor='w', edgecolor='k')
step = 100
plt.scatter(ra[::step, ::step], dec[::step, ::step])
plt.grid()
plt.set_xticks(np.arange(0, 100, 20))
plt.set_yticks(np.arange(0, 100, 20))

plt.show()



#pixelIDs = hp.ang2pix(NSIDE, ra, dec, lonlat=True)
#print (pixelIDs)

#dict_table = dict(zip(pixelIDs, fits_data))
#print (dict_table)






#pixelID = hp.ang2pix(NSIDE, ra, dec, lonlat=True)
#ipix_image[index[0]*index[1]] = pixelID
    

#np.array()

#print (pixel)


In [ ]:

print ("NPIX: %d" %NPIX)
m = np.arange(NPIX)
print (m)

vec = hp.ang2vec(np.pi / 2, np.pi * 3 / 4)
print(vec)

ipix_disc = hp.query_disc(nside=32, vec=vec, radius=np.radians(10))
m = np.arange(NPIX)
m[ipix_disc] = m.max()
hp.mollview(m, nest=True, title="Mollview image RING")

In [ ]:
query1 = r"""insert into cube values marray it in [0:0, 0:0, 0:0, 0:0, 0:0] values {0.0f, 0.0f} """;
query2 = "update cube as c set c assign shift(decode($1), [%f, %f])" %

print(fits_data)
try:
    db_connector.open()
    query_executor.execute_write(query1)    
    query_executor.execute_write(query2, fits_data)
finally:
    db_connector.close()

In [ ]:
print (type(fits_data))
print(fits_data.shape)




plt.imshow(fits_data, cmap='gray', norm=LogNorm())
plt.colorbar()

In [ ]:
fits_subset = fits_data[850:900, 1350:1450]

plt.imshow(fits_subset, cmap='gray', norm=LogNorm())
plt.colorbar()